In [8]:
##### prepare data used for GPSnet
import pandas as pd
import numpy as np
import math
import scipy.io
import copy
from scipy import stats
# import matplotlib.pyplot as plt
import os

In [6]:
### User provided trait list
# traits_list=['updrs2','updrs3','schwab',#motor
#              'updrs1','updrs4','tremor_scores','pigd_scores',
#              'moca','benton', 'hvlt_delayed_recall','hvlt_recog_disc_index',#cognition
#              'hvlt_retention','hvlt_total_recall', 'lns','semantic_fluency','symbol_digit',#cognition
#              'gds','stai',#mood
#              'scopa',#automatic
#              'rem','ess',#sleep
#              'quip','gco',
#              'alpha_syn','total_tau','abeta_42','p_tau181p'#biomarker
#              ]

traits_list=['updrs3']


In [15]:
lmm_result_path='./lmm_result/PD_combined/per_trait/'
file_suffix='_linear_mt.csv'
version='genetic_PD_combined'

In [4]:
# reference files
gene2snp_ref_path='../../ref/combined_map_gene2snp.csv'
gene_id_ref_path='../../ref/gene_vocab.csv'
gene2snp_ref_df=pd.read_csv(gene2snp_ref_path)
gene_id_ref_df=pd.read_csv(gene_id_ref_path)

In [19]:
# create folder
save_data_path='../GPSnet/'+version+'/data/'
save_code_path='../GPSnet/'+version+'/code/'
if not os.path.exists(save_data_path):
    os.makedirs(save_data_path)

if not os.path.exists(save_code_path):
    os.makedirs(save_code_path)

In [20]:
def prep_mat(trait,save_data_path):
    df=pd.read_csv(lmm_result_path+trait+file_suffix)

    ## select SNPs whose p_mt < 0.05 and split its genes
    sig_df_list=[]#save gene, p, beta
    for i in range(df.shape[0]):
        one_line=df.iloc[i,]
        p_mt=one_line['p_mt']
        if p_mt < 0.05:
            gene_str=one_line['Genes']
            if pd.isnull(gene_str):#no matched gene 
                print('no matched gene for %s' % one_line['SNP'])
            else:
                if '|' in gene_str:
                    genes=gene_str.split('|')
                else:#one gene
                    genes=gene_str
                for g in genes:
                    sig_df_list.append([g,one_line['p_mt'],one_line['beta_mt']])
    sig_df=pd.DataFrame(sig_df_list,columns=['GENE','PVAL','BETA'])
    
    ## map gene name/ensembl ID to NCBI id
    id_df=[]
    for i in range(sig_df.shape[0]):
        gene=sig_df.iloc[i,0]
        pval=sig_df.iloc[i,1]
        beta=sig_df.iloc[i,2]
        if gene.startswith('ENSG'):
            one_line_ref = gene_id_ref_df.loc[gene_id_ref_df.ensembl_id==gene]
        else:
            one_line_ref=gene_id_ref_df.loc[gene_id_ref_df.symbol==gene]
        if len(one_line_ref)>0:#if one gene has multiple NCBI ids
            ncbi_id=one_line_ref['ncbi_id']
            for j in ncbi_id.values:
                id_df.append([j,pval,beta])
        else:
            pass
            #print('no one_line_ref:'+gene)#[PVRL2,SLC12A2-DT,DYNLRB2-AS1,FLJ33630,MS4,TRIM51G,SIPA1L1-AS1]
    id_df = pd.DataFrame(id_df, columns=['ncbi_id', 'PVAL', 'BETA'])
    
    # select the largest -logp*beta for each gene id
    all_gene_list=list(set(list(id_df.ncbi_id)))
    input_df=[]
    for g in all_gene_list:
        sub_df=id_df.loc[id_df['ncbi_id']==g]
        v_max=0
        for i_sub in range(sub_df.shape[0]):
            v=-math.log10(sub_df.iloc[i_sub,:]['PVAL']) * abs(sub_df.iloc[i_sub,:]['BETA'])
            if v > v_max:
                v_max=v
        input_df.append([g, v_max])
    input_df = pd.DataFrame(input_df, columns=['ncbi_id', 'sig'])
    
    # normalize value
    norm_value=stats.zscore(input_df['sig'])#z-score
    norm_value=(norm_value-np.min(norm_value))/(np.max(norm_value)-np.min(norm_value))#min-max
    norm_df=copy.deepcopy(input_df)
    norm_df['sig']=norm_value
    
    scipy.io.savemat(save_data_path+trait+'_linear.mat', {'Mutation':np.array(norm_df)})
    
    return norm_df


In [21]:
all_gene_id_list=[]
for trait in traits_list:
    ### linear model results
    linear_df=prep_mat(trait,save_data_path)
    print('finish '+trait)
    #print('max in %s: %f' % (trait,linear_df.sig.max()))
    gene_id_list = list(linear_df.ncbi_id)
    all_gene_id_list+=gene_id_list

#### prepare gene length mat, set 1 for all genes
all_gene_id_list=list(set(all_gene_id_list))
gene_length_list=[1]*len(all_gene_id_list)
gene_length_df=pd.DataFrame({'ncbi_id':all_gene_id_list,'length':gene_length_list})
id_mat=np.array(gene_length_df)
scipy.io.savemat(save_data_path+'gene_length_df.mat', {'Gene_Length':id_mat})

##### prepare matlab code
for trait in traits_list:
    out=open(save_code_path+'/run_'+trait+'.m','w')
    out.write("trait='%s';\n"%trait)
    out.write("Raw_Module_Generation(trait,0.5,'linear');\n")
    # out.write("ppmi_gene=Cancer_Module_Calculation(trait,0.5,0.005,'linear');\n")
    # out.write("save_f = ['../result/',trait,'.txt'];\n")
    # out.write("f = fopen(save_f, 'w');\n")
    # out.write("fprintf(f, '%d\\n', ppmi_gene);\n")
    # out.write("fclose(f);")
    out.close()

finish updrs3
max in updrs3: 1.000000


In [22]:
# copy essential data/code
cmd1=f'cp ../GPSnet/ref_data/PPI.mat {save_data_path}'
cmd2=f'cp ../GPSnet/ref_code/matlab/* {save_code_path}'
os.system(cmd1)
os.system(cmd2)

0